In [4]:
#####
#Note
#####
#There is more functionality in the draft version - this is just a simple working version that's organised

#Uncomment to install the following packages
#%pip install googletrans==4.0.0-rc1 
#%pip install unidecode
#%pip install langdetect
#%pip install regex

#APIs and Libraries used for transaltion
from googletrans import Translator
from langdetect import detect
from unidecode import unidecode

#Imports
import os
import sys
import time
import regex
import threading

#Initialised translator
translator = Translator()
translator.raise_Exception = True

def runScript():
    path = "C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy"  #getDirPath()
    logPath = path.rsplit("\\", 1)[0] + "\\EngToAsiLangTransLog.txt"
    #threadOne = threading.Thread(name="Rename Process 1", target=trasnlateFilesInDirectoy, args=(path, logPath)).start()
    trasnlateFilesInDirectoy(path, logPath)

def trasnlateFilesInDirectoy(path, logPath):
    fullFilePaths, rootOfPaths, nameOfFiles = listFilePaths(path)
    translatedFileNames = translateList(nameOfFiles, logPath) #The 2nd arguements specifies where to write the text files showing all the before and after translations
    translatedFileNames = resolveDuplicates(translatedFileNames)

    print("## Renaming files with translations ##")
    print("######################################\n")
    i = 0
    for fullP, rootP, tranlatedFN in zip(fullFilePaths, rootOfPaths, translatedFileNames):
        #Formats translations
        formatedTranlatedFN = upperAndLowerCaseFormat(tranlatedFN)

        #Renames files using translations
        os.rename(fullP, os.path.join(rootP, formatedTranlatedFN))
        #Prints progress
        print("[" + str(i + 1) + "/" + str(len(fullFilePaths)) + "]: " + str(os.path.join(rootP, formatedTranlatedFN)))
        i += 1
    
    print("> Done")

#################
#HELPER FUNCTIONS
#################

#Gets and formats the path of the directory that contains the MP3 files (and sub directories with MP3 files) that we want to rename with translations
def getDirPath():
    #Asks the user for a path to do the operations on
    path = input("File Path: ")

    #If there is no backslash in the path, the bath is not in the windows user path and the path doesn't exist, a new path is requested
    while "\\" not in path and ":\\Users" not in path and os.path.isdir(path):
        print("File Path Invalid")
        path = input("Please Select A Different File Path: ")
    #Gets rid of trailing backslashes or any file names attached to the end of the path
    if path.split("\\")[-1] == "" or "." in path.split("\\")[-1]:
        path = path.rsplit("\\", 1)[0]

#Gets, stores and returns the a list of filenames, parent directories and full paths or all the files in the given path and it's sub directories 
def listFilePaths(path):
    #These hold the filenames and the directories they're contained within seperarely and together
    fullFilePaths = [] 
    rootOfPaths = [] 
    nameOfFiles = []

    #Recursively goes through all directories and finds files within them
    for root, directories, files in os.walk(path):
        for file in files:
            fullFilePaths.append(os.path.join(root, file)) #The full path for each file is stored here
            nameOfFiles.append(file.rsplit(".", 1)[0]) #The names of each file found is stored here - the file extension is removes so it is not transalted
            rootOfPaths.append(root) #The path to each file's parent directory is stored her - this is prepended to the transalted filename later
    
    return fullFilePaths, rootOfPaths, nameOfFiles

#Translates list of filesnames
def translateList(untranslatedList, logPath):
    #checks if the script needs to be resumed and loads the data needed to resume if it does
    untranslatedList_resumed, translatedFileNames_resumed, resumeIndex = resumeFromLastSession(logPath)
    if resumeIndex == -1 or untranslatedList_resumed == None:
        translatedFileNames = []
        translationNo = 1
        if os.path.exists(logPath):
            os.remove(logPath)
        print(logBehaviour("## Start Of Logs ##\n###################\n", logPath))
    else:
        untranslatedList = untranslatedList_resumed
        translatedFileNames = translatedFileNames_resumed
        translationNo = resumeIndex
        print(logBehaviour("\n## Start Of Logs - Execution Re-Attempt ##\n##########################################\n", logPath))
    
    #Trasnlates filenames to english
    for i in range(translationNo -1, len(untranslatedList)):
        print(logBehaviour("\n> ", logPath).replace("\n", ""), end="")
        time.sleep(0.1) #Puts the script to sleep inbetween translations to avoid HTTP 429 errors from sending too many requests at once
        
        #If filename is already fully in english, it is not traslated
        untranslatedName = untranslatedList[i].replace("\n", "")
        translation = untranslatedList[i]
        if isEnglish(translation):
            translatedFileNames.append(untranslatedName)
        else:
            #Attempts to translate text
            try: 
                translation = translateText(untranslatedName)
                translatedFileNames.append(translation)
            #A 429 error could occur when the input string translates to an output string that is too large to send back - this crops the input text
            except TypeError:
                translatedFileNames = handleTypeError(untranslatedList, translatedFileNames, untranslatedName, logPath, translationNo)
                translation = translatedFileNames[-1]
                continue
            #A 429 error could occur if too many requests are made at once, so when that happens, the program is put to sleep (takes a short break) and then resumes
            except:
                translatedFileNames = handle429Reponse(untranslatedList, translatedFileNames, untranslatedName, logPath, translationNo)
                translation = translatedFileNames[-1]
                continue

        print(logBehaviour("[" + str(translationNo) + "/" + str(len(untranslatedList)) + "]: " + untranslatedName + "  \n\t\t===>  " + (translation if translation != untranslatedName else "[[Same as before - no translation]]"), logPath))
        translationNo += 1
    return translatedFileNames

#Translates the text to english - the source language is specified as japanese if hiragana and katakana strings are found in the string
def translateText(textToTranslate):
    if hasHiraAndKata(textToTranslate):
        translation = translator.translate(textToTranslate[:250], src="ja", dest="en").text[:250]
    #elif hasNoHiraOrKata(textToTranslate):
        #return translator.translate(textToTranslate[:250], src="zh-cn", dest="en").text[:250]
        #return translator.translate(textToTranslate[:250], src="zh-tw", dest="en").text[:250]
    else:
        translation = translator.translate(textToTranslate[:250], dest="en").text[:250] 

    return removeFirstWordIfLentivularBrackets(translation.replace("\n", ""))

#Checks for duplicates in the list of filename translations and resolves them by adding a suffix - which makes it unique
def resolveDuplicates(uncheckedList):
    duplicates = []
    for item in uncheckedList:
        if uncheckedList.count(item) > 1:
            #Makes note of duplicates
            duplicates.append(item)

    #Makes duplicates list distinct - removes duplicates from it
    duplicates = list(set(duplicates))

    #Handles duplicates if any are found
    if duplicates:
        #Goes through each duplicate name found and results all the duplicates with the name
        for dup in duplicates:
            #Character to within in the suffix "-[]" - which is appended to duplicates
            duplicateSuffixChar = "I"
            duplicateSuffix = "-[]"

            suffixRepeater = 1 #Keeps track of the amount of "I"s we want in the "-[]" suffix 

            #Gives each ducplidate item a suffix
            for i in range(len(uncheckedList)):
                if uncheckedList[i] == dup:
                    duplicateSuffix = duplicateSuffix[:2] + duplicateSuffixChar + duplicateSuffix[2:] #Adds 'I' within suffix before appending to the filename 
                    suffixRepeater += 1 #Insures the suffix will have an extra "I" when the next duplicate is found
                    uncheckedList[i] = uncheckedList[i] + duplicateSuffix #Adds suffix to translated file
                    
        #Checks if the duplicate handler introduced duplicates
        return resolveDuplicates(uncheckedList)
    else:
        return uncheckedList

#If the scipted ended abuptly in the last run, the log files will have data that this function can extract to allow the program to resume
def resumeFromLastSession(logPath):
    #Checks log file exists
    if os.path.exists(logPath):
        logFileContents = []

        #Reads contents of log files
        with open(logPath, encoding='utf8') as file:
            logFileContents = file.readlines()
            
        #Makes sure the log file has information to resume stored
        if len(logFileContents) >= 3:
            initialLogs = logFileContents[:-3] #Normal logs
            lastLogs = logFileContents[-4:] #Logs that indicate an abrupt and the state of the program during it

            if "Script ended abruptly!" in lastLogs[0]:
                #Deletes log file to recreate them without the error logs at the end
                os.remove(logPath)
                for line in initialLogs:
                    logBehaviour(line, logPath)
                
                #Extracts, formats and splits data from the log file needed to resume
                fUntransatedFilenameList = lastLogs[2].split("-[?/?]-")[:-1].copy()
                fUntransatedFilenameList.append(lastLogs[2].split("-[?/?]-")[-1:][0].rsplit("\n", 1)[0])
                translatedFilenameList = lastLogs[3].split("-[?/?]-")
                translationProcessIndex = int(lastLogs[0][lastLogs[0].find('[')+1:lastLogs[0].find(']')].split(" ")[2])

                #returns the untranslated list, translated list and the index the program left off on
                return (fUntransatedFilenameList, translatedFilenameList, translationProcessIndex)
                
    return [], [], -1 #Returns nothing if there was no log file of if the file did not have the expected data
 
#checks if a string is fully english
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

#Checks if the text has hiragana or katakana (or specail japanese characters like "々" - e.g. hito'bito')
def hasHiraAndKata(s):
    return True if regex.search("([ぁ-ゔ]|[ぁ-ヴー]|[々〆〤])", s) != None else False

#Checks if string has CJK (chinese, japanese, korean) or alphanumic characters in it and returns true if it does !!!Haven't made sure it works in most situations
def hasCJKCharacters(s, **kwargs):
    if kwargs.get('mode', None) == "HiraKataOnly":
        return True if regex.search("([\p{Hiragana}\p{Katakana}])+", s) != None else False

    return True if regex.search("([\p{Hiragana}\p{Katakana}\p{Han}])+", s) != None else False
    
#Checks if there are chinese, korean or Japanese kanji (I.E. Hanzi) characters in the text - returns true if there isn't !!!Haven't made sure it works in most situations
def hasNoHanzi(s):
    return True if regex.search("(([\p{Hiragana}\p{Katakana}\p{Han}]*[\p{Hiragana}\p{Katakana}]+[\p{Hiragana}\p{Katakana}\p{Han}]*)|[a-zA-Z0-9]|[ａ-ｚＡ-Ｚ０-９])+", input) != None else False

#Checks if the text has hiragana or katakana, returns true is there isn't any !!!Haven't made sure it works in most situations
def hasNoHiraOrKata(s):
    #False if re.search("^(【)(.)*([\p{Hiragana}\p{Katakana}])+(.)*(】)$", input) != None else True
    return False if regex.search("([\p{Hiragana}\p{Katakana}])", s) != None else True

#Replaces specail characters not allowed for fine names to ones that are and makes every word start with a capital and proceed with lowercase
def upperAndLowerCaseFormat(s):
    s = (s
        .replace("\"", "'")
        .replace("?", "？")
        .replace(":", "﹕")
        .replace("\\", "⧵")
        .replace("/", "∕")
        .replace("*", "✲")
        .replace("<", "〈")
        .replace(">", "〉")
        .replace("|", "｜")
        .replace("‘", "'")
        .replace("\n", "")[:250] 
    )

    formattedString = ""

    for word in s.split(" "):
        formattedString += word[:1].upper() + word[1:].lower() + " "
        
    return formattedString.rsplit(" ", 1)[0] + ".mp3"

#Remmoves leading spaces and first occurance of text with that starts and ends with "【】" (Lentivular)
def removeFirstWordIfLentivularBrackets(s):
   pattern = regex.compile("^( *)((【[一-龠]+】)|(【ドラマＣＤ】)|(【[dD][Rr][aA][mM][aA]】)|(【[dD][rR][aA][mM][aA] [cCＣ][dDＤ]】)|(【广播剧】)|(【[rR][aA][dD][iI][oO]】))( *)", regex.UNICODE)
   return pattern.sub('', s)

#Logs messages to text file
def logBehaviour(textToLog, logPath):
    with open(logPath, 'a+', encoding='utf8') as file: 
        file.write(textToLog)
        file.close()

    return textToLog

#Attempts to handle typeError exceptions
def handleTypeError(untranslatedList, translatedFileNames, fileName, logPath, translationNo):
    pattern = regex.compile("(【[一-龠]+】)|(【ドラマＣＤ】)|(【[dD][Rr][aA][mM][aA]】)|(【[dD][rR][aA][mM][aA] [cCＣ][dDＤ]】)|(广播剧)|(【广播剧】)|(【[rR][aA][dD][iI][oO]】)", regex.UNICODE)

    try:
        formattedUntranslatedText = pattern.sub('', fileName)
        croppedLen = len(fileName) - len(formattedUntranslatedText)
        if len(formattedUntranslatedText) >= len(fileName) or croppedLen < 5:
            alreadyCropped = croppedLen if croppedLen < 5 else 0
            if len(formattedUntranslatedText) > 30:
                formattedUntranslatedText = formattedUntranslatedText[:-(20-alreadyCropped)] + "(crpd)"
            else:
                formattedUntranslatedText = formattedUntranslatedText[:-int(len(formattedUntranslatedText)/4)] + "(crpd)"
        translation = translateText(formattedUntranslatedText)
        translatedFileNames.append(translation)
        print(logBehaviour("[" + str(translationNo) + "/" + str(len(untranslatedList)) + "]: " + fileName + "  \n\t\t===>  " + translation, logPath))
        translationNo += 1
    except TypeError:
        exceptionMessage = "Failed to resolve type Error."
        logStateBeforeExit(untranslatedList, translatedFileNames, translationNo, logPath, exceptionMessage)
    except:
        exceptionMessage = "A HTTP 429 Error Occured Again!"
        logStateBeforeExit(untranslatedList, translatedFileNames, translationNo, logPath, exceptionMessage)

    return translatedFileNames


#Handles 429 HTTP Reponse error by retrying and aborting if it fails again
def handle429Reponse(untranslatedList, translatedFileNames, fileName, logPath, translationNo):
    noOfTicks = 5 #12
    sleepLen = 3 #5
    print(
        " Error.\n\n" +
        "> A HTTP 429 Error Occured. Too many translation requests were sent to the google API in a short period of time. " +
        "Pausing script and resuming in about " + ((str(int((noOfTicks * sleepLen)/60)) + " minutes") if ((noOfTicks * sleepLen) > 60) else (str(noOfTicks * sleepLen) + " seconds")) + "to attempt to rectify issue. \n> " + 
        "Note that if the script end with an error/exception again, the daily qouta of trasnlation the google API allows has likely been reached." + 
        "Note that none of these translations have been used to rename anything yet.\n" +
        "[", 
    end="")
    for i in range(noOfTicks):
        time.sleep(sleepLen)
        print("#", end="")
    print("]\n>> Attempting to Resume! <<\n> ", end="")

    try:
        translation = translateText(fileName)
        translatedFileNames.append(translation)
        print(logBehaviour("[" + str(translationNo) + "/" + str(len(untranslatedList)) + "]: " + fileName + "  \n\t\t===>  " + translation, logPath))
        translationNo += 1
    except:
        exceptionMessage = "A HTTP 429 Error Occured Again!"
        logStateBeforeExit(untranslatedList, translatedFileNames, translationNo, logPath, exceptionMessage)

    return translatedFileNames

def logStateBeforeExit(untranslatedList, translatedFileNames, translationNo, logPath, exceptionMessage):
    #Saves list of untranslated filename (in the order they were going to be translated) and the index that the program was on before siezing translation and ending
    logBehaviour("Error.\n\n#####  Script ended abruptly! [Index = " + str(translationNo) + "] <<List of all filenames and all trasnalted filenames below>>  ######\n" + "\n", logPath) #Log index and indication of abrupt end
    fileList = ""
    translatedFileList = ""

    #Log untranslated list
    for item in untranslatedList[:]:
        fileList += str(item) + "-[?/?]-"
    logBehaviour((fileList.rsplit("-[?/?]-", 1)[0] + "\n"), logPath)

    #Log translated file list
    for item in translatedFileNames[:]:
        translatedFileList += str(item) + "-[?/?]-"
    logBehaviour((translatedFileList.rsplit("-[?/?]-", 1)[0]), logPath)

    #Present used with error
    print("Failed. Aborting Script.")
    raise Exception(str(exceptionMessage) + " Aborting Script. Try running the script again, the scipt will resume from where it left off in this session.")


def reRunScriptOnException(path, logPath):
    try:
        trasnlateFilesInDirectoy(path, logPath)
    except:
        print("> The Script has reran, resumed and failed twice. Ending Script.")
        raise Exception("The Script has reran, resumed and failed twice. Ending Script.")

##########################
#Main function placeholder
##########################
sys.tracebacklimit = 0
runScript()


## Start Of Logs - Execution Re-Attempt ##
##########################################

> [539/943]: 【化物語】 ドラマCD 『佰物語』 100編のお題について 阿良々木暦 と 各ヒロイン達が語り合う！  
		===>  Drama CD "Talk of Tale" About the topic of 100 Hen Tree and Each heroine talks about!
> [539/943]: 【哥布林杀手】GA文库 14卷特装版 广播剧CD「スワッピング・スワンプフィーバー」  
		===>  【Corporation for forest championship】 GA Bunka 14 Gomi Special Edition Shrimp CD "Swapping Swump Fever"
> [540/943]: 【哥布林杀手】GA文库小说12卷特装版 广播剧CD「インタビュー・ウィズ・ザ・アドベンチャラーズ」  
		===>  [Corporation forest remembers] GA Bunkota 12 Gae Special Edition Shrimp CD "Interview With The Adventurers"
> [541/943]: 【哥布林杀手】剧场版 广播剧CD 哥杀 蜥蜴僧侣 矿人道士篇  
		===>  [布林 killer] Theatrical version of the radio drama CD brother kill the lizard monk mineral
> [542/943]: 【哥布林杀手】剧场版 广播剧CD 牧牛妹 柜台小姐 魔女篇  
		===>  [Goblin killer] Theatrical version of the radio drama CD animal husbandry counters Miss Women
> [543/943]: 【字幕 井口裕香】《爱书的下克上》·侍从的本分 （神殿巫女篇2 第三章）  
		===>  [Subtitle Jingkou Yuxiang] "The next King of Love"